In [71]:
from timeit import default_timer as timer
from sklearn import metrics
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import os
import requests
import warnings
from IPython.display import HTML
warnings.filterwarnings('ignore')

In [72]:
df = pd.read_csv("Ford car price - Prediction.csv")
df.head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,Fiesta,2017,12000,Automatic,15944,Petrol,150,57.7,1.0
1,Focus,2018,14000,Manual,9083,Petrol,150,57.7,1.0
2,Focus,2017,13000,Manual,12456,Petrol,150,57.7,1.0
3,Fiesta,2019,17500,Manual,10460,Petrol,145,40.3,1.5
4,Fiesta,2019,16500,Automatic,1482,Petrol,145,48.7,1.0


In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17966 entries, 0 to 17965
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         17966 non-null  object 
 1   year          17966 non-null  int64  
 2   price         17966 non-null  int64  
 3   transmission  17966 non-null  object 
 4   mileage       17966 non-null  int64  
 5   fuelType      17966 non-null  object 
 6   tax           17966 non-null  int64  
 7   mpg           17966 non-null  float64
 8   engineSize    17966 non-null  float64
dtypes: float64(2), int64(4), object(3)
memory usage: 1.2+ MB


In [74]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [75]:
df["transmission"] = le.fit_transform(df["transmission"])
df["fuelType"] = le.fit_transform(df["fuelType"])

In [76]:
df.columns

Index(['model', 'year', 'price', 'transmission', 'mileage', 'fuelType', 'tax',
       'mpg', 'engineSize'],
      dtype='object')

In [77]:
x = df[['transmission', 'mileage', 'fuelType', 'tax',
       'mpg', 'engineSize']]
y = df['price']

In [78]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=0)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((16169, 6), (1797, 6), (16169,), (1797,))

In [79]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
#ss = MinMaxScaler()
ss= StandardScaler()

In [80]:
x_train = ss.fit_transform(x_train)
x_test = ss.transform(x_test)

In [81]:
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [82]:
from sklearn.linear_model import LinearRegression

params = {
    "n_jobs": -1,
    "copy_X": False
}
start = timer()
model = LinearRegression(**params).fit(x_train, y_train)
train_patched = timer() - start
f"Intel® extension for Scikit-learn time: {train_patched:.2f} s"

'Intel® extension for Scikit-learn time: 0.00 s'

In [83]:
y_predict = model.predict(x_test)
mse_metric_opt = metrics.mean_squared_error(y_test, y_predict)
f'Patched Scikit-learn MSE: {mse_metric_opt}'

'Patched Scikit-learn MSE: 9098961.610905876'

In [84]:
from sklearnex import unpatch_sklearn
unpatch_sklearn()

In [85]:
from sklearn.linear_model import LinearRegression

start = timer()
model = LinearRegression(**params).fit(x_train, y_train)
train_unpatched = timer() - start
f"Original Scikit-learn time: {train_unpatched:.2f} s"

'Original Scikit-learn time: 0.01 s'

In [86]:
y_predict = model.predict(x_test)
mse_metric_original = metrics.mean_squared_error(y_test, y_predict)
f'Original Scikit-learn MSE: {mse_metric_original}'

'Original Scikit-learn MSE: 9098961.610905873'

In [87]:
HTML(f"<h3>Compare MSE metric of patched Scikit-learn and original</h3>"
     f"MSE metric of patched Scikit-learn: {mse_metric_opt} <br>"
     f"MSE metric of unpatched Scikit-learn: {mse_metric_original} <br>"
     f"Metrics ratio: {mse_metric_opt/mse_metric_original} <br>"
     f"<h3>With Scikit-learn-intelex patching you can:</h3>"
     f"<ul>"
     f"<li>Use your Scikit-learn code for training and prediction with minimal changes (a couple of lines of code);</li>"
     f"<li>Fast execution training and prediction of Scikit-learn models;</li>"
     f"<li>Get the similar quality</li>"
     f"<li>Get speedup in <strong>{(train_unpatched/train_patched):.1f}</strong> times.</li>"
     f"</ul>")